In [1]:
import requests
import pandas as pd
from API_Key import api_key

pages = list(range(1,11))

popular = pd.DataFrame()

for page in pages:
    movieData = pd.DataFrame(requests.get(f'https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=en-US&page={page}').json()['results'])
    popular = popular.append(movieData)


popular = popular[popular['adult'] == False]

In [2]:
popular_features = popular[['id','title','release_date','popularity','vote_average','vote_count']]
popular_features = popular_features.rename(columns={
                                        "id":"movie_id",
                                        "popularity":"movie_popularity"
                                        })

In [3]:
id_list = popular_features['movie_id'].tolist()
revenue_list = []
budget_list = []

for movie in id_list:
    x = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['revenue']
    y = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['budget']
    revenue_list.append(x)
    budget_list.append(y)

revenue_df = pd.DataFrame(
    {'movie_id':id_list,
    'budget': budget_list,
    'revenue':revenue_list
    })

revenue_df.reset_index(drop = True)

,movie_id,budget,revenue
0,566525,150000000,430238384
1,370172,242000000,708000000
2,580489,110000000,442000000
3,512195,200000000,0
4,585245,0,22000000
...,...,...,...
195,567189,0,0
196,857702,0,0
197,10138,200000000,623933331
198,839100,0,0


In [12]:

Movie_Data = popular_features.merge(revenue_df, on='movie_id', how='inner')
Movie_Data.count()

movie_id            200
title               200
release_date        200
movie_popularity    200
vote_average        200
vote_count          200
budget              200
revenue             200
dtype: int64

In [30]:
ratings = []
x = 0
for movie_id in Movie_Data['movie_id']:
    movie_rating = requests.get(f'http://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&append_to_response=release_dates').json()['release_dates']['results']
    for rating in movie_rating:
        if rating['iso_3166_1'] == 'US':
            try:
                ratings.append(rating['release_dates'][1]["certification"])
                x=1
            except IndexError:
                x=1
                ratings.append(rating['release_dates'][0]["certification"])
    if x == 0:
        ratings.append('')
    x=0

In [32]:
Movie_Data['rating'] = ratings
Movie_Data

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating
0,566525,Shang-Chi and the Legend of the Ten Rings,2021-09-01,7750.478,7.9,3181,150000000,430238384,PG-13
1,370172,No Time to Die,2021-09-29,4212.055,7.6,1927,242000000,708000000,PG-13
2,580489,Venom: Let There Be Carnage,2021-09-30,4295.722,6.8,2008,110000000,442000000,PG-13
3,512195,Red Notice,2021-11-04,5461.940,6.9,1051,200000000,0,PG-13
4,585245,Clifford the Big Red Dog,2021-11-10,3574.383,7.9,150,0,22000000,PG
...,...,...,...,...,...,...,...,...,...
195,567189,Tom Clancy's Without Remorse,2021-04-29,175.781,7.2,1661,0,0,R
196,857702,LEGO Star Wars Terrifying Tales,2021-10-01,181.491,6.7,101,0,0,G
197,10138,Iron Man 2,2010-04-28,183.229,6.8,16967,200000000,623933331,PG-13
198,839100,Bright: Samurai Soul,2021-10-12,193.701,6.1,23,0,0,PG-13


In [34]:
filtered_table = Movie_Data[Movie_Data['budget']!= 0]
filtered_table = filtered_table[filtered_table['revenue'] != 0]
filtered_movies = filtered_table[filtered_table['rating'] != '']
filtered_movies

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating
0,566525,Shang-Chi and the Legend of the Ten Rings,2021-09-01,7750.478,7.9,3181,150000000,430238384,PG-13
1,370172,No Time to Die,2021-09-29,4212.055,7.6,1927,242000000,708000000,PG-13
2,580489,Venom: Let There Be Carnage,2021-09-30,4295.722,6.8,2008,110000000,442000000,PG-13
9,524434,Eternals,2021-11-03,1822.535,7.1,1072,200000000,300000000,PG-13
10,438631,Dune,2021-09-15,1635.364,8.0,4085,165000000,354264482,PG-13
...,...,...,...,...,...,...,...,...,...
188,615457,Nobody,2021-03-18,196.628,8.3,3676,17000000,61779301,R
191,293660,Deadpool,2016-02-09,173.026,7.6,25788,58000000,783100000,R
192,198663,The Maze Runner,2014-09-10,187.018,7.2,14017,34000000,348319861,PG-13
194,271110,Captain America: Civil War,2016-04-27,187.647,7.4,18860,250000000,1153296293,PG-13


In [37]:
filtered_movies.count()

movie_id            77
title               77
release_date        77
movie_popularity    77
vote_average        77
vote_count          77
budget              77
revenue             77
rating              77
dtype: int64

In [39]:

titleNid = filtered_movies[['movie_id','title']]

titleNid = titleNid.reset_index(drop = True)

titleNid.to_csv('Tables/titleNid.csv',index = False)

filtered_movies.to_csv('Tables/Movie_Data.csv')

In [ ]:
genre = requests.get(f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=en-US').json()['genres']
genre_df = pd.DataFrame(genre)
genre_df
genre_df.to_csv('Tables/Genres.csv')